# Data cleaning

In [16]:
import json
import pandas as pd

In [4]:
d = json.load(open("data/2021_11_10_16_46_Board_94B97E6BA7B0_PowerOnOff_1_i3pwya3wka1j43oc_File_0.bmerawdata"))

In [13]:
d["rawDataBody"]["dataColumns"]

[{'name': 'Sensor Index',
  'unit': '',
  'format': 'integer',
  'key': 'sensor_index'},
 {'name': 'Sensor ID', 'unit': '', 'format': 'integer', 'key': 'sensor_id'},
 {'name': 'Time Since PowerOn',
  'unit': 'Milliseconds',
  'format': 'integer',
  'key': 'timestamp_since_poweron'},
 {'name': 'Real time clock',
  'unit': 'Unix Timestamp: seconds since Jan 01 1970. (UTC); 0 = missing',
  'format': 'integer',
  'key': 'real_time_clock'},
 {'name': 'Temperature',
  'unit': 'DegreesClecius',
  'format': 'float',
  'key': 'temperature'},
 {'name': 'Pressure',
  'unit': 'Hectopascals',
  'format': 'float',
  'key': 'pressure'},
 {'name': 'Relative Humidity',
  'unit': 'Percent',
  'format': 'float',
  'key': 'relative_humidity'},
 {'name': 'Resistance Gassensor',
  'unit': 'Ohms',
  'format': 'float',
  'key': 'resistance_gassensor'},
 {'name': 'Heater Profile Step Index',
  'unit': '',
  'format': 'integer',
  'key': 'heater_profile_step_index'},
 {'name': 'Scanning enabled',
  'unit': '',


In [25]:
data = pd.DataFrame(d["rawDataBody"]["dataBlock"])
data.columns = [item['key'] for item in d["rawDataBody"]["dataColumns"]]

In [26]:
data.describe()

,sensor_index,sensor_id,timestamp_since_poweron,real_time_clock,temperature,pressure,relative_humidity,resistance_gassensor,heater_profile_step_index,scanning_enabled,label_tag,error_code
count,18400.00000,1.840000e+04,1.840000e+04,1.840000e+04,18400.000000,18400.000000,18400.000000,1.840000e+04,18400.000000,18400.000000,18400.000000,18400.0
mean,3.50000,1.619189e+09,3.659113e+06,1.636566e+09,36.072545,1014.772184,26.117367,4.981060e+06,4.500000,0.980000,1.234402,0.0
std,2.29135,2.842064e+08,2.141301e+06,2.141515e+03,2.010091,0.316654,3.094530,9.435927e+06,2.872359,0.140004,0.835904,0.0
min,0.00000,1.334977e+09,3.274000e+03,1.636563e+09,29.371206,1014.051208,20.457424,3.967762e+04,0.000000,0.000000,0.000000,0.0
25%,1.75000,1.334992e+09,1.805364e+06,1.636565e+09,34.599586,1014.520081,23.558842,1.830533e+05,2.000000,1.000000,0.000000,0.0
50%,3.50000,1.619189e+09,3.659064e+06,1.636566e+09,36.027401,1014.768982,25.777097,4.440590e+05,4.500000,1.000000,1.000000,0.0
75%,5.25000,1.903391e+09,5.514242e+06,1.636568e+09,37.741463,1014.993057,27.875109,4.790294e+06,7.000000,1.000000,2.000000,0.0
max,7.00000,1.903395e+09,7.323845e+06,1.636570e+09,41.912571,1015.629333,39.547005,6.884034e+07,9.000000,1.000000,2.000000,0.0


Zostawiamy: temperature, pressure, relative_humidity, resistance_gassensor

Czy zostawić heater_profile_step_index?

In [32]:
X = data[["temperature", "pressure", "relative_humidity", "resistance_gassensor"]]
y = data["label_tag"]

In [48]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)
clf = MLPClassifier(hidden_layer_sizes=(100,100,100,), activation='tanh').fit(X_train, y_train)

clf.score(X_test, y_test)

0.49326086956521736

mamy acc małpy rzucającej monetą

## Wariant jeśli pliki są pożar/nie-pożar

In [57]:
def clean_file(filename, label):
    """
    reads and cleans json from file and adds column with label
    """
    
    d = json.load(open(filename))
    data = pd.DataFrame(d["rawDataBody"]["dataBlock"])
    data.columns = [item['key'] for item in d["rawDataBody"]["dataColumns"]]
    X = data[["temperature", "pressure", "relative_humidity", "resistance_gassensor"]]
    X.loc[:, 'label'] = label
    return X

In [61]:
pozar = clean_file("./data/2021_11_10_16_46_Board_94B97E6BA7B0_PowerOnOff_1_i3pwya3wka1j43oc_File_0.bmerawdata", 0)
niepozar = clean_file("./data/2021_11_11_17_59_Board_94B97E6BA7B0_PowerOnOff_1_u6zwow6t2f25togi_File_0.bmerawdata", 1)

<ipython-input-57-f9919705820d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, 'label'] = label
<ipython-input-57-f9919705820d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.loc[:, 'label'] = label


In [66]:
data = pd.concat([pozar, niepozar])

In [71]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

y = data[["label"]]
X = data[["temperature", "pressure", "relative_humidity", "resistance_gassensor"]]

X_train, X_test, y_train, y_test = train_test_split(X, y)
clf = MLPClassifier(hidden_layer_sizes=(32,32,), activation='logistic').fit(X_train, y_train)

clf.score(X_test, y_test)

/home/hania/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1118: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6865493587831792

In [75]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.6865493587831792
